In [ ]:
import os
import json
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay
)
from google.colab import files

In [ ]:
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d dhoogla/unswnb15 -q
!unzip -q unswnb15.zip

Dataset URL: https://www.kaggle.com/datasets/dhoogla/unswnb15
License(s): CC-BY-NC-SA-4.0


In [ ]:
!kaggle datasets download -d aryashah2k/nfuqnidsv2-network-intrusion-detection-dataset -q
!unzip -q nfuqnidsv2-network-intrusion-detection-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/aryashah2k/nfuqnidsv2-network-intrusion-detection-dataset
License(s): CC0-1.0


In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
unsw = pd.read_parquet('UNSW_NB15_training-set.parquet')

# Drop unnecessary columns
unsw.drop(columns=[c for c in ['srcip', 'dstip', 'sport', 'dsport', 'stime', 'ltime'] if c in unsw.columns],
           inplace=True)

# Label encode categorical features
for col in ['proto', 'service', 'state']:
    if col in unsw.columns:
        le = LabelEncoder()
        unsw[col] = le.fit_transform(unsw[col].astype(str))

# Handle missing labels and encode attack labels
unsw['attack_cat'] = unsw['attack_cat'].fillna('Normal')
le_attack = LabelEncoder()
unsw['label_encoded'] = le_attack.fit_transform(unsw['attack_cat'])

# Drop original label column
if 'attack_cat' in unsw.columns:
    unsw.drop(columns=['attack_cat'], inplace=True)

# Replace inf/nan
unsw = unsw.replace([float('inf'), -float('inf')], np.nan).dropna()

# Scale numeric features
numeric_cols_unsw = unsw.select_dtypes(include=['int64','float64']).columns.tolist()
numeric_cols_unsw = [c for c in numeric_cols_unsw if c != 'label_encoded']
scaler = MinMaxScaler()
unsw[numeric_cols_unsw] = scaler.fit_transform(unsw[numeric_cols_unsw])

# Save client 1
unsw.to_csv("/content/client1_UNSW.csv", index=False)
print("UNSW dataset saved as client1_UNSW.csv")

UNSW dataset saved as client1_UNSW.csv


In [ ]:
nfq = pd.read_csv("NF-UQ-NIDS-v2.csv", nrows=200000)
nfq = nfq.sample(n=175341, random_state=42).reset_index(drop=True)

# Drop unnecessary columns
nfq.drop(columns=[c for c in ['Flow ID', 'Source IP', 'Destination IP',
                              'Src Port', 'Dst Port', 'Timestamp',
                              'StartTime', 'EndTime'] if c in nfq.columns],
         inplace=True)

# Label encode categorical features
for col in ['proto', 'service', 'state']:
    if col in nfq.columns:
        le = LabelEncoder()
        nfq[col] = le.fit_transform(nfq[col].astype(str))

# Encode attack labels
nfq['Attack'] = nfq['Attack'].replace('Benign', 'Normal').fillna('Normal')
base_mapping = dict(zip(le_attack.classes_, le_attack.transform(le_attack.classes_)))
current_max = max(base_mapping.values())
for att in nfq['Attack'].unique():
    if att not in base_mapping:
        current_max += 1
        base_mapping[att] = current_max
nfq['label_encoded'] = nfq['Attack'].map(base_mapping).astype(int)

# Drop original attack column
if 'Attack' in nfq.columns:
    nfq.drop(columns=['Attack'], inplace=True)

if 'FLOW_DURATION_MILLISECONDS' in nfq.columns:
    nfq['FLOW_DURATION_MILLISECONDS'] = nfq['FLOW_DURATION_MILLISECONDS'] / 1000

mapping = {
    'FLOW_DURATION_MILLISECONDS': 'dur',
    'PROTOCOL': 'proto',
    'IN_PKTS': 'spkts',
    'OUT_PKTS': 'dpkts',
    'IN_BYTES': 'sbytes',
    'OUT_BYTES': 'dbytes',
    'SRC_TO_DST_AVG_THROUGHPUT': 'sload',
    'DST_TO_SRC_AVG_THROUGHPUT': 'dload',
    'RETRANSMITTED_IN_BYTES': 'sloss',
    'RETRANSMITTED_OUT_BYTES': 'dloss',
    'LONGEST_FLOW_PKT': 'sinpkt',
    'SHORTEST_FLOW_PKT': 'dinpkt',
    'TCP_WIN_MAX_OUT': 'dwin',
    'TCP_WIN_MAX_IN': 'swin',
    'IPV4_SRC_ADDR': 'srcip',
    'L4_SRC_PORT': 'srcport',
    'IPV4_DST_ADDR':'dstip',
    'L4_DST_PORT': 'dstport',
    'L7_PROTO': 'service',
    'TCP_FLAGS': 'state',
}

nfq.rename(columns=mapping, inplace=True)

# Keep only columns that exist in UNSW + label_encoded
common_features = [c for c in unsw.columns if c != 'label_encoded' and c in nfq.columns]

# Drop columns in NFQ that are not mapped
nfq = nfq[common_features + ['label_encoded']]

# Scale numeric features
numeric_cols_nfq = nfq.select_dtypes(include=['int64','float64']).columns.tolist()
numeric_cols_nfq = [c for c in numeric_cols_nfq if c != 'label_encoded']
nfq[numeric_cols_nfq] = scaler.fit_transform(nfq[numeric_cols_nfq])

# Save client 2
nfq.to_csv("/content/client2_NFQ.csv", index=False)
print("NF-UQ dataset saved as client2_NFQ.csv")

NF-UQ dataset saved as client2_NFQ.csv


In [ ]:
# Define broader unified attack categories
attack_mapping_unified = {
    # Normal traffic
    'Normal': 'Normal',
    'Benign': 'Normal',

    # Denial of Service
    'DoS': 'DoS',
    'DDoS': 'DoS',
    'DoS GoldenEye': 'DoS',
    'DoS Hulk': 'DoS',
    'DoS Slowhttptest': 'DoS',
    'DoS Slowloris': 'DoS',
    'Service DoS': 'DoS',

    # Scanning & Reconnaissance
    'Reconnaissance': 'Reconnaissance',
    'PortScan': 'Reconnaissance',
    'Scanning': 'Reconnaissance',
    'Fuzzers': 'Reconnaissance',

    # Exploits / Injection
    'Exploits': 'Exploits',
    'Shellcode': 'Exploits',
    'Worms': 'Exploits',
    'Backdoor': 'Exploits',
    'Generic': 'Exploits',
    'SQL Injection': 'Exploits',
    'Command Injection': 'Exploits',
    'Code Injection': 'Exploits',

    # Information Theft
    'Theft': 'Theft',
    'Data Exfiltration': 'Theft',
    'Data Theft': 'Theft',
    'Information Gathering': 'Theft',

    # Web-based Attacks
    'Web Attack': 'Web Attack',
    'Brute Force': 'Web Attack',
    'Cross Site Scripting': 'Web Attack',
    'XSS': 'Web Attack',
    'Infiltration': 'Web Attack',

    # Generic Malware
    'Trojan': 'Malware',
    'Virus': 'Malware',
    'Botnet': 'Malware',
    'Malware': 'Malware',
}

if 'label_encoded' in unsw.columns:
    # Need to recover original label names temporarily
    unsw_labels = le_attack.inverse_transform(unsw['label_encoded'])
    unsw['UnifiedAttack'] = [attack_mapping_unified.get(a, 'Other') for a in unsw_labels]

if 'label_encoded' in nfq.columns:
    # If NFQ was mapped via base_mapping, get inverse
    inv_base_mapping = {v: k for k, v in base_mapping.items()}
    nfq_labels = [inv_base_mapping.get(a, 'Normal') for a in nfq['label_encoded']]
    nfq['UnifiedAttack'] = [attack_mapping_unified.get(a, 'Other') for a in nfq_labels]

all_labels = list(set(unsw['UnifiedAttack'].unique()).union(set(nfq['UnifiedAttack'].unique())))
le_unified = LabelEncoder()
le_unified.fit(all_labels)

unsw['label_encoded'] = le_unified.transform(unsw['UnifiedAttack'])
nfq['label_encoded'] = le_unified.transform(nfq['UnifiedAttack'])

# Update base mapping with unified labels
base_mapping = dict(zip(le_unified.classes_, le_unified.transform(le_unified.classes_)))

print("Unified Attack Mapping:")
for k, v in base_mapping.items():
    print(f"{v}: {k}")

# Drop helper columns
unsw.drop(columns=['UnifiedAttack'], inplace=True)
nfq.drop(columns=['UnifiedAttack'], inplace=True)


Unified Attack Mapping:
0: DoS
1: Exploits
2: Normal
3: Other
4: Reconnaissance
5: Theft
6: Web Attack


In [ ]:
# Count samples per attack type in both datasets
unsw_counts = unsw['label_encoded'].value_counts().rename_axis('label_encoded').reset_index(name='UNSW_count')
nfq_counts = nfq['label_encoded'].value_counts().rename_axis('label_encoded').reset_index(name='NFQ_count')

# Merge counts by label
attack_counts = pd.merge(unsw_counts, nfq_counts, on='label_encoded', how='outer').fillna(0)

# Map label codes to class names using base_mapping
inv_base_mapping = {v: k for k, v in base_mapping.items()}
attack_counts['Attack_Type'] = attack_counts['label_encoded'].map(inv_base_mapping)

# Add combined total count
attack_counts['Total_Samples'] = attack_counts['UNSW_count'] + attack_counts['NFQ_count']

# Reorder columns for readability
attack_counts = attack_counts[['Attack_Type', 'label_encoded', 'UNSW_count', 'NFQ_count', 'Total_Samples']]

# Sort by total samples (descending)
attack_counts = attack_counts.sort_values(by='Total_Samples', ascending=False).reset_index(drop=True)

print("\n===== Attack Label Distribution Across Datasets =====")
print(attack_counts.to_string(index=False))
print("=====================================================")



===== Attack Label Distribution Across Datasets =====
   Attack_Type  label_encoded  UNSW_count  NFQ_count  Total_Samples
        Normal              2     56000.0      57609       113609.0
           DoS              0     12264.0      91913       104177.0
      Exploits              1     76402.0        143        76545.0
Reconnaissance              4     28675.0       6103        34778.0
         Other              3      2000.0      19272        21272.0
    Web Attack              6         0.0        298          298.0
         Theft              5         0.0          3            3.0


In [ ]:
# 1 Remove 'Theft' samples if present
if 'Theft' in base_mapping:
    theft_label = base_mapping['Theft']

    # Filter out Theft samples
    unsw = unsw[unsw['label_encoded'] != theft_label].reset_index(drop=True)
    nfq = nfq[nfq['label_encoded'] != theft_label].reset_index(drop=True)

    print(f"Removed 'Theft' attacks (label code = {theft_label}) from both datasets.")
else:
    print("No 'Theft' label found — skipping removal.")

# 2 Reindex remaining labels
# Extract all unique labels from both datasets
unique_labels = sorted(list(set(unsw['label_encoded'].unique()) | set(nfq['label_encoded'].unique())))

# Create a new sequential mapping (0,1,2,...)
new_label_mapping = {old: new for new, old in enumerate(unique_labels)}

# Apply new mapping
unsw['label_encoded'] = unsw['label_encoded'].map(new_label_mapping)
nfq['label_encoded'] = nfq['label_encoded'].map(new_label_mapping)

# 3 Update base_mapping to reflect new indices
inv_base_mapping = {v: k for k, v in base_mapping.items()}
base_mapping = {inv_base_mapping[old]: new for old, new in new_label_mapping.items() if old in inv_base_mapping}

# 4 Print updated label summary
print("\n Final Unified Attack Mapping (after removing Theft & reindexing):")
for k, v in base_mapping.items():
    print(f"{v}: {k}")

# 5 Optional: Display new label counts
print("\nUpdated label distribution:")
unsw_counts = unsw['label_encoded'].value_counts().sort_index()
nfq_counts = nfq['label_encoded'].value_counts().sort_index()
for lbl in sorted(base_mapping.values()):
    attack_name = [k for k, v in base_mapping.items() if v == lbl][0]
    print(f"{attack_name:<15} | UNSW: {unsw_counts.get(lbl,0):<6} | NFQ: {nfq_counts.get(lbl,0):<6}")


Removed 'Theft' attacks (label code = 5) from both datasets.

✅ Final Unified Attack Mapping (after removing Theft & reindexing):
0: DoS
1: Exploits
2: Normal
3: Other
4: Reconnaissance
5: Web Attack

Updated label distribution:
DoS             | UNSW: 12264  | NFQ: 91913 
Exploits        | UNSW: 76402  | NFQ: 143   
Normal          | UNSW: 56000  | NFQ: 57609 
Other           | UNSW: 2000   | NFQ: 19272 
Reconnaissance  | UNSW: 28675  | NFQ: 6103  
Web Attack      | UNSW: 0      | NFQ: 298   


MIA FOR DECISION TREE (TRADITIONAL ML MODEL)

In [ ]:
from sklearn.model_selection import train_test_split
combined_df = pd.concat([unsw, nfq], axis=0).reset_index(drop=True)
X = combined_df.drop(columns=['label_encoded'])
y = combined_df['label_encoded']

print("Feature count:", X.shape[1])
print("Unique classes:", y.nunique())
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

Feature count: 35
Unique classes: 6


In [ ]:
uploaded_model = files.upload()

os.makedirs("uploaded_models", exist_ok=True)
for fname, bytestr in uploaded_model.items():
    path = os.path.join("uploaded_models", fname)
    with open(path, "wb") as f:
        f.write(bytestr)
    print(f"Saved: {path}")
model = None
preferred = os.path.join("uploaded_models", "decision_tree_model.pkl")
if os.path.exists(preferred):
    model = joblib.load(preferred)
    print(f"Loaded model from {preferred}")
else:
    for f in os.listdir("uploaded_models"):
        if f.lower().endswith((".pkl", ".joblib")):
            model = joblib.load(os.path.join("uploaded_models", f))
            print(f"Loaded model from uploaded_models/{f}")
            break

if model is None:
    raise FileNotFoundError("No .pkl/.joblib model file found in uploaded files.")

print("Model type:", type(model))


Saving decision_tree_model.pkl to decision_tree_model (1).pkl
Saved: uploaded_models/decision_tree_model (1).pkl
Loaded model from uploaded_models/decision_tree_model.pkl
Model type: <class 'sklearn.tree._classes.DecisionTreeClassifier'>


In [ ]:
X_mem_val, X_mem_eval, y_mem_val, y_mem_eval = train_test_split(
    X_train, y_train, test_size=0.5, random_state=42, stratify=y_train
)
X_nonmem_val, X_nonmem_eval, y_nonmem_val, y_nonmem_eval = train_test_split(
    X_test, y_test, test_size=0.5, random_state=42, stratify=y_test
)

In [ ]:
def true_class_confidence(model_obj, X, y_true):
    if hasattr(model_obj, "predict_proba"):
        proba = model_obj.predict_proba(X)
        y_true = np.clip(y_true, 0, proba.shape[1]-1)
        return proba[np.arange(len(y_true)), y_true]
    else:
        preds = model_obj.predict(X)
        return (preds == y_true).astype(float)

In [ ]:
scores_mem_val = true_class_confidence(model, X_mem_val, y_mem_val)
scores_nonmem_val = true_class_confidence(model, X_nonmem_val, y_nonmem_val)

all_scores = np.concatenate([scores_mem_val, scores_nonmem_val])
labels = np.concatenate([np.ones_like(scores_mem_val), np.zeros_like(scores_nonmem_val)])

fpr, tpr, thresholds = roc_curve(labels, all_scores)
balanced_acc = (tpr + (1 - fpr)) / 2
best_threshold = thresholds[np.argmax(balanced_acc)]

print(f" Optimal threshold found: {best_threshold:.4f}")

 Optimal threshold found: 0.5000


In [ ]:
scores_mem_eval = true_class_confidence(model, X_mem_eval, y_mem_eval)
scores_nonmem_eval = true_class_confidence(model, X_nonmem_eval, y_nonmem_eval)

pred_mem_eval = (scores_mem_eval >= best_threshold).astype(int)
pred_nonmem_eval = (scores_nonmem_eval >= best_threshold).astype(int)

true_mem_eval = np.ones_like(pred_mem_eval)
true_nonmem_eval = np.zeros_like(pred_nonmem_eval)

pred_all = np.concatenate([pred_mem_eval, pred_nonmem_eval])
true_all = np.concatenate([true_mem_eval, true_nonmem_eval])

ASR = accuracy_score(true_all, pred_all)
privacy_score = 1 - ASR
attack_auc = roc_auc_score(true_all, np.concatenate([scores_mem_eval, scores_nonmem_eval]))

print("\n--- Membership Inference Attack Results ---")
print(f"Attack Success Rate (ASR): {ASR:.4f}")
print(f"Privacy Score (1 - ASR): {privacy_score:.4f}")
print(f"Attack AUC: {attack_auc:.4f}")


--- Membership Inference Attack Results ---
Attack Success Rate (ASR): 0.7191
Privacy Score (1 - ASR): 0.2809
Attack AUC: 0.5013


In [ ]:
os.makedirs("mia_results", exist_ok=True)

# Save results
results = {
    "ASR": float(ASR),
    "privacy_score": float(privacy_score),
    "attack_auc": float(attack_auc),
    "threshold": float(best_threshold),
}
with open("mia_results/mia_results.json", "w") as f:
    json.dump(results, f, indent=2)

# Plot ROC curve
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, lw=2, label=f"AUC={attack_auc:.3f}")
plt.plot([0,1],[0,1],'--',color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("MIA ROC Curve")
plt.legend()
plt.tight_layout()
plt.savefig("mia_results/mia_roc_curve.png", dpi=300)
plt.close()

# Histogram of member vs non-member scores
plt.figure(figsize=(7,5))
plt.hist(scores_mem_eval, bins=40, alpha=0.6, label="Members")
plt.hist(scores_nonmem_eval, bins=40, alpha=0.6, label="Non-Members")
plt.axvline(best_threshold, color='k', linestyle='--', label='Threshold')
plt.legend()
plt.title("Confidence Distribution: Members vs Non-Members")
plt.tight_layout()
plt.savefig("mia_results/mia_score_histogram.png", dpi=300)
plt.close()

# Confusion matrix for attack predictions
cm = confusion_matrix(true_all, pred_all)
ConfusionMatrixDisplay(cm, display_labels=["Non-Member","Member"]).plot(cmap="Blues")
plt.title("Attack Confusion Matrix")
plt.tight_layout()
plt.savefig("mia_results/mia_confusion_matrix.png", dpi=300)
plt.close()

# Save everything as zip for download
import shutil
shutil.make_archive("mia_results", "zip", "mia_results")
files.download("mia_results.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

MIA FOR FEDAVG MODEL


In [ ]:
# Re-encode labels
le = LabelEncoder()
all_labels = pd.concat([unsw['label_encoded'], nfq['label_encoded']])
le.fit(all_labels)
unsw['label_encoded'] = le.transform(unsw['label_encoded'])
nfq['label_encoded'] = le.transform(nfq['label_encoded'])

In [ ]:
# Normalize features per client
common_features = [c for c in unsw.columns if c != 'label_encoded' and c in nfq.columns]
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
X1 = pd.DataFrame(scaler1.fit_transform(unsw[common_features]), columns=common_features)
X2 = pd.DataFrame(scaler2.fit_transform(nfq[common_features]), columns=common_features)
y1 = unsw['label_encoded'].copy()
y2 = nfq['label_encoded'].copy()

In [ ]:
import random
import tensorflow as tf
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# -----------------------------
# 2. Stratified Train/Test Split
# -----------------------------
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
X1_np, y1_np = X1.values.astype('float32'), y1.values.astype('int32')
X2_np, y2_np = X2.values.astype('float32'), y2.values.astype('int32')

train_idx1, test_idx1 = next(sss.split(X1_np, y1_np))
train_idx2, test_idx2 = next(sss.split(X2_np, y2_np))

X1_train, X1_test = X1_np[train_idx1], X1_np[test_idx1]
y1_train, y1_test = y1_np[train_idx1], y1_np[test_idx1]
X2_train, X2_test = X2_np[train_idx2], X2_np[test_idx2]
y2_train, y2_test = y2_np[train_idx2], y2_np[test_idx2]

num_classes = len(le.classes_)

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report


In [ ]:
from google.colab import files

uploaded_fedavg = files.upload()
for fn in uploaded_fedavg.keys():
  if fn.endswith(".h5"):
    model_path = fn

model = load_model(model_path)
print(f"Loaded model from: {model_path}")

Saving global_model_full.h5 to global_model_full (2).h5


Loaded model from: global_model_full (2).h5


In [ ]:
def privacy_predict_probs(model, X,
                          top_k=None,
                          round_ndigits=None,
                          rng_seed=None):

    if rng_seed is not None:
        np.random.seed(rng_seed)

    preds = model.predict(X, verbose=0)
    preds = np.asarray(preds)

    if preds.ndim == 1:
        probs_pos = preds.ravel()
        probs = np.vstack([1 - probs_pos, probs_pos]).T
    elif preds.shape[1] == 1:
        probs_pos = preds.ravel()
        probs = np.vstack([1 - probs_pos, probs_pos]).T
    else:
        probs = preds
        if np.any(probs < 0) or np.max(probs) > 1.0001 or np.min(probs) < -0.0001:
            exp = np.exp(probs - np.max(probs, axis=1, keepdims=True))
            probs = exp / np.sum(exp, axis=1, keepdims=True)

    if top_k is not None:
        k = int(top_k)
        if k < probs.shape[1]:
            idx = np.argsort(probs, axis=1)[:, ::-1]  # descending indices
            mask = np.zeros_like(probs, dtype=bool)
            rows = np.arange(probs.shape[0])[:, None]
            topk_idx = idx[:, :k]
            mask[rows, topk_idx] = True
            probs = probs * mask.astype(float)

    if round_ndigits is not None:
        nd = int(round_ndigits)
        probs = np.round(probs, decimals=nd)

    row_sums = probs.sum(axis=1, keepdims=True)
    zero_rows = (row_sums.squeeze() == 0)
    if np.any(zero_rows):
        probs[zero_rows, :] = 1.0 / probs.shape[1]
        row_sums = probs.sum(axis=1, keepdims=True)
    probs = probs / row_sums

    return probs

In [ ]:
import zipfile
OUT_ROOT = "/content/mia_results_fedavg"
ZIP_PATH = os.path.join(OUT_ROOT, "mia_results_clients.zip")

In [ ]:
def analyze_client(model, X_train, X_test, y_train, y_test, client_name="client"):
    import os, json
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.metrics import (
        roc_curve, accuracy_score, roc_auc_score,
        confusion_matrix, classification_report
    )

    # ---- Create output directory safely ----
    outdir = os.path.join("/content/mia_results_fedavg", f"mia_results_{client_name}")
    os.makedirs(outdir, exist_ok=True)

    # ---- Get model confidence scores ----
    def get_probs(m, X):
        preds = m.predict(X, verbose=0)
        preds = np.asarray(preds)
        if preds.ndim == 1:
            probs_pos = preds.ravel()
            probs = np.vstack([1 - probs_pos, probs_pos]).T
        elif preds.shape[1] == 1:
            probs_pos = preds.ravel()
            probs = np.vstack([1 - probs_pos, probs_pos]).T
        else:
            probs = preds
            if np.any(probs < 0) or np.max(probs) > 1.0001 or np.min(probs) < -0.0001:
                exp = np.exp(probs - np.max(probs, axis=1, keepdims=True))
                probs = exp / np.sum(exp, axis=1, keepdims=True)
        return probs

    probs_train = get_probs(model, X_train)
    probs_test  = get_probs(model, X_test)
    conf_train = np.max(probs_train, axis=1)
    conf_test  = np.max(probs_test, axis=1)

    y_member_train = np.ones(len(conf_train), dtype=int)
    y_member_test  = np.zeros(len(conf_test), dtype=int)
    conf_all = np.concatenate([conf_train, conf_test])
    y_member_all = np.concatenate([y_member_train, y_member_test])

    # ---- ROC & best threshold ----
    fpr, tpr, thresholds = roc_curve(y_member_all, conf_all)
    youden = tpr - fpr
    best_idx = np.argmax(youden)
    best_thresh = thresholds[best_idx]

    attack_preds = (conf_all >= best_thresh).astype(int)
    ASR = accuracy_score(y_member_all, attack_preds)
    AUC = roc_auc_score(y_member_all, conf_all)
    cm = confusion_matrix(y_member_all, attack_preds, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()

    member_acc = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    nonmember_acc = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    privacy_score = 1.0 - ASR

    # ---- Save classification report ----
    clr_text = classification_report(y_member_all, attack_preds, zero_division=0)
    clr = classification_report(y_member_all, attack_preds, output_dict=True, zero_division=0)
    with open(os.path.join(outdir, "classification_report.txt"), "w") as f:
        f.write(clr_text)
    pd.DataFrame(clr).transpose().to_csv(os.path.join(outdir, "classification_report.csv"))

    # ---- Save summary JSON ----
    summary = {
        "client": client_name,
        "best_threshold": float(best_thresh),
        "AUC": float(AUC),
        "ASR": float(ASR),
        "privacy_score": float(privacy_score),
        "TP": int(tp), "FP": int(fp), "FN": int(fn), "TN": int(tn),
        "member_TPR": float(member_acc),
        "nonmember_TNR": float(nonmember_acc)
    }
    with open(os.path.join(outdir, "summary.json"), "w") as f:
        json.dump(summary, f, indent=2)

    # ---- Save ROC plot ----
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC={AUC:.4f}")
    plt.plot([0,1],[0,1],'--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve - {client_name}")
    plt.legend()
    plt.savefig(os.path.join(outdir, "roc_curve.png"))
    plt.close()
    print(summary)

    print(f" Results saved for {client_name} → {outdir}")
    return outdir, summary


In [ ]:
clients_dirs = []
out1, sum1 = analyze_client(model, X1_train, X1_test, y1_train, y1_test, client_name="client1")
out2, sum2 = analyze_client(model, X2_train, X2_test, y2_train, y2_test, client_name="client2")

import zipfile, os
zip_path = "/content/mia_results_fedavg/mia_results_clients.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for d in [out1, out2]:
        for root, _, files in os.walk(d):
            for f in files:
                full = os.path.join(root, f)
                arc = os.path.relpath(full, "/content/mia_results_fedavg")
                zf.write(full, arc)
print(" All results zipped at:", zip_path)


{'client': 'client1', 'best_threshold': 0.8181066513061523, 'AUC': 0.5016818320198441, 'ASR': 0.5660969197164383, 'privacy_score': 0.4339030802835617, 'TP': 85371, 'FP': 21180, 'FN': 54901, 'TN': 13889, 'member_TPR': 0.6086104140526977, 'nonmember_TNR': 0.3960477914967635}
 Results saved for client1 → /content/mia_results_fedavg/mia_results_client1
{'client': 'client2', 'best_threshold': 0.7643875479698181, 'AUC': 0.4989476834216375, 'ASR': 0.6338044234564099, 'privacy_score': 0.36619557654359014, 'TP': 101352, 'FP': 25290, 'FN': 38918, 'TN': 9778, 'member_TPR': 0.7225493690739289, 'nonmember_TNR': 0.2788297022926885}
 Results saved for client2 → /content/mia_results_fedavg/mia_results_client2
 All results zipped at: /content/mia_results_fedavg/mia_results_clients.zip


In [ ]:
from google.colab import files
files.download("/content/mia_results_fedavg/mia_results_clients.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

MIA FOR FEDPROX

In [ ]:
from google.colab import files
uploaded_fedprox = files.upload()

for fn in uploaded_fedprox.keys():
    if fn.endswith(".h5"):
        model_path = fn

from tensorflow.keras.models import load_model
model = load_model(model_path)
print(f"Loaded model from: {model_path}")


Saving global_model_full.h5 to global_model_full (1).h5
Loaded model from: global_model_full (1).h5


In [ ]:
OUT_ROOT = "/content/mia_results_fedprox"
ZIP_PATH = os.path.join(OUT_ROOT, "mia_results_clients.zip")

In [ ]:
def analyze_client_privacy(model, X_train, X_test, y_train, y_test,
                           client_name="client",
                           clip_value=0.95, noise_std=0.0, top_k=None, round_ndigits=None,
                           rng_seed=None, out_root=OUT_ROOT):

    outdir = os.path.join("/content/mia_results_fedprox", f"mia_results_{client_name}")
    os.makedirs(outdir, exist_ok=True)

    probs_train = privacy_predict_probs(model, X_train,
                                        clip_value=clip_value,
                                        noise_std=noise_std,
                                        top_k=top_k,
                                        round_ndigits=round_ndigits,
                                        rng_seed=rng_seed)
    probs_test  = privacy_predict_probs(model, X_test,
                                        clip_value=clip_value,
                                        noise_std=noise_std,
                                        top_k=top_k,
                                        round_ndigits=round_ndigits,
                                        rng_seed=rng_seed+1 if rng_seed is not None else None)

    conf_train = np.max(probs_train, axis=1)
    conf_test  = np.max(probs_test, axis=1)

    y_member_train = np.ones(len(conf_train), dtype=int)
    y_member_test  = np.zeros(len(conf_test), dtype=int)

    conf_all = np.concatenate([conf_train, conf_test])
    y_member_all = np.concatenate([y_member_train, y_member_test])

    # ROC and threshold
    fpr, tpr, thresholds = roc_curve(y_member_all, conf_all)
    youden = tpr - fpr
    best_idx = np.argmax(youden)
    best_thresh = thresholds[best_idx]

    attack_preds = (conf_all >= best_thresh).astype(int)
    ASR = accuracy_score(y_member_all, attack_preds)
    AUC = roc_auc_score(y_member_all, conf_all)
    cm = confusion_matrix(y_member_all, attack_preds, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    privacy_score = 1.0 - ASR

    # Save classification report
    clr_text = classification_report(y_member_all, attack_preds, zero_division=0)
    clr = classification_report(y_member_all, attack_preds, output_dict=True, zero_division=0)
    with open(os.path.join(outdir, "classification_report.txt"), "w") as f:
        f.write(clr_text)
    pd.DataFrame(clr).transpose().to_csv(os.path.join(outdir, "classification_report.csv"))

    # Save summary
    summary = {
        "client": client_name,
        "n_members_train": int(len(conf_train)),
        "n_nonmembers_test": int(len(conf_test)),
        "best_threshold": float(best_thresh),
        "AUC": float(AUC),
        "ASR": float(ASR),
        "privacy_score": float(privacy_score),
        "clip_value": clip_value,
        "noise_std": noise_std,
        "top_k": top_k,
        "round_ndigits": round_ndigits,
        "TP": int(tp), "FP": int(fp), "FN": int(fn), "TN": int(tn)
    }
    with open(os.path.join(outdir, "summary.json"), "w") as f:
        json.dump(summary, f, indent=2)

    # ROC plot
    plt.figure()
    plt.plot(fpr, tpr)
    plt.plot([0,1],[0,1],'--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC (AUC={AUC:.4f}) - {client_name}")
    plt.savefig(os.path.join(outdir, "roc_curve.png"))
    plt.close()

    # Privacy score curve across thresholds
    thr_grid = np.linspace(np.min(conf_all)-1e-6, np.max(conf_all)+1e-6, 200)
    asr_list = [accuracy_score(y_member_all, (conf_all >= t).astype(int)) for t in thr_grid]
    privacy_curve = 1.0 - np.array(asr_list)
    pd.DataFrame({"threshold": thr_grid, "ASR": asr_list, "privacy_score": privacy_curve}).to_csv(os.path.join(outdir, "privacy_curve_values.csv"), index=False)
    plt.figure()
    plt.plot(thr_grid, privacy_curve)
    plt.xlabel("Confidence threshold")
    plt.ylabel("Privacy score (1 - ASR)")
    plt.title(f"Privacy curve - {client_name}")
    plt.savefig(os.path.join(outdir, "privacy_score_curve.png"))
    plt.close()

    # Confidence histograms
    plt.figure()
    plt.hist(conf_train, bins=50)
    plt.title(f"Confidence hist - members (train) - {client_name}")
    plt.savefig(os.path.join(outdir, "conf_hist_train.png"))
    plt.close()

    plt.figure()
    plt.hist(conf_test, bins=50)
    plt.title(f"Confidence hist - non-members (test) - {client_name}")
    plt.savefig(os.path.join(outdir, "conf_hist_test.png"))
    plt.close()
    print(summary)

    print(f"Saved privacy-aware MIA results for {client_name} -> {outdir}")
    return outdir, summary

In [ ]:
PRIVACY_PARAMS = {
    "clip_value": 0.90,
    "noise_std": 0.005,
    "top_k": 3,
    "round_ndigits": 3,
    "rng_seed": 234
}

clients_dirs = []
out1, sum1 = analyze_client_privacy(model, X1_train, X1_test, y1_train, y1_test,
                                    client_name="client1",
                                    **PRIVACY_PARAMS)
clients_dirs.append(out1)

out2, sum2 = analyze_client_privacy(model, X2_train, X2_test, y2_train, y2_test,
                                    client_name="client2",
                                    **PRIVACY_PARAMS)
clients_dirs.append(out2)
import zipfile, os
# Zip results
zip_path = "/content/mia_results_fedprox/mia_results_clients.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for d in clients_dirs:
        for root, _, files in os.walk(d):
            for fn in files:
                full = os.path.join(root, fn)
                arc = os.path.relpath(full, OUT_ROOT)
                zf.write(full, arc)

{'client': 'client1', 'n_members_train': 140272, 'n_nonmembers_test': 35069, 'best_threshold': 0.5830784913353722, 'AUC': 0.5024186755935535, 'ASR': 0.5971221790682156, 'privacy_score': 0.40287782093178437, 'clip_value': 0.9, 'noise_std': 0.005, 'top_k': 3, 'round_ndigits': 3, 'TP': 92574, 'FP': 22943, 'FN': 47698, 'TN': 12126}
Saved privacy-aware MIA results for client1 -> /content/mia_results_fedprox/mia_results_client1
{'client': 'client2', 'n_members_train': 140270, 'n_nonmembers_test': 35068, 'best_threshold': 0.9867986798679867, 'AUC': 0.5009744612406443, 'ASR': 0.43900922789127284, 'privacy_score': 0.5609907721087272, 'clip_value': 0.9, 'noise_std': 0.005, 'top_k': 3, 'round_ndigits': 3, 'TP': 55670, 'FP': 13763, 'FN': 84600, 'TN': 21305}
Saved privacy-aware MIA results for client2 -> /content/mia_results_fedprox/mia_results_client2


In [ ]:
from google.colab import files
files.download("/content/mia_results_fedprox/mia_results_clients.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>